In [1]:
import os
from xml.dom.minidom import Document,parse
from collections import defaultdict
import re
import numpy as np
import pandas as pd
import cv2 as cv
import time

In [2]:
class OpeateXML:

    def __init__(self, srcPath: str, targetPath: str, srcFileName: str):
        self._srcPath = srcPath
        self._targetPath = targetPath
        self._srcFileName = srcFileName

    def readSrcFileName(self, fileEncoding="utf8") -> defaultdict:
        data = defaultdict(list)
        s = re.compile("\.AIpng_\d{1,}", re.IGNORECASE)
        srcFileFullPath = os.path.join(self._srcPath, self._srcFileName)
        try:
            if os.path.exists(srcFileFullPath):
                with open(srcFileFullPath, mode="r", encoding=fileEncoding, errors="ignore") as fr:
                    for content in fr.readlines():
                        data[s.sub(".AIpng",content.strip().split(",")[0])].append(content.strip())
        except Exception as ex:
#             MyLogger().error(f"OperateXML:read file error:\n{ex}")
            print(f"OperateXML:read file error:\n{ex}")
            return {}
        else:
            # data.sort(key=lambda x: x.strip().split(",")[0])
            return data

    def getCreateXMLData(self,srcData:dict,mnlData:list)->defaultdict:
        """
        获取手动确认的图片
        srcData:Location.txt中的原始数据
        mnlData:手动确认数据
        """
        try:
            for key,values in srcData.items():
                for item in mnlData:
                    for v in values:
                        if item in v.strip().split(",")[0]:
                            srcData[key][srcData[key].index(v)]=srcData[key][srcData[key].index(v)].replace("auto","mnl")
        except Exception as ex:
#             MyLogger().error(f"OperateXML: get data from location and mnldata interaction error\n{ex}")
            print(f"OperateXML: get data from location and mnldata interaction error\n{ex}")
            return {}
        else:
            return srcData

    def operateXML(self,data:defaultdict)->None:
        for k in data.keys():
            xmlFileFullPath = os.path.join(self._targetPath, os.path.splitext(k)[0]+".xml")
            if os.path.exists(xmlFileFullPath):
                self.appendExistXML(data={k:data[k]},xmlFileFullPath=xmlFileFullPath)
            else:
                self.createNewXML({k:data[k]})

    def appendExistXML(self,data:defaultdict,xmlFileFullPath:str,fileEncoding="utf8"):
        try:
            doc = parse(xmlFileFullPath)
            rootNode = doc.documentElement
            # print(rootNode.nodeName)
            key=rootNode.getElementsByTagName("filename")[0].childNodes[0].data
            objs=rootNode.getElementsByTagName("object")
            for obj in objs:
                name=obj.getElementsByTagName("name")[0].childNodes[0].data
                bndboxs = obj.getElementsByTagName("bndbox")
                for bndbox in bndboxs:
                    xmin = bndbox.getElementsByTagName("xmin")[0].childNodes[0].data
                    ymin = bndbox.getElementsByTagName("ymin")[0].childNodes[0].data
                    xmax = bndbox.getElementsByTagName("xmax")[0].childNodes[0].data
                    ymax = bndbox.getElementsByTagName("ymax")[0].childNodes[0].data
                existData=f"existData,{ymin},{xmin},{ymax},{xmax},{name}"
                data[key].append(existData)
            data[key]=list(set(data[key]))
            os.remove(xmlFileFullPath)
            self.createNewXML(data=data)
        except Exception as ex:
#             MyLogger().error(f"OperateXML:append content to {xmlFileFullPath} error\n{ex}")
            print(f"OperateXML:append content to {xmlFileFullPath} error\n{ex}")
            return

    def createNewXML(self, data: dict, fileEncoding="utf8")->None:
        """
        data:传入的数据字典
        fileEncoding：XML默认编码格式
        """
        if data:
            try:
                for k,v in data.items():
                    doc = Document()
                    # 创建根节点
                    rootNode = doc.createElement("annotation")
                    # 添加根节点
                    doc.appendChild(rootNode)

                    folder = doc.createElement("folder")
                    folderText = doc.createTextNode(self._targetPath)
                    folder.appendChild(folderText)
                    rootNode.appendChild(folder)

                    filename = doc.createElement("filename")
                    filenameText = doc.createTextNode(k)
                    filename.appendChild(filenameText)
                    rootNode.appendChild(filename)

                    path = doc.createElement("path")
                    pathText = doc.createTextNode(os.path.join(self._targetPath,k))
                    path.appendChild(pathText)
                    rootNode.appendChild(path)
                    for i in v:
                        tmpData = i.strip().split(",")
                        if len(tmpData) == 6:
                            _, ymin, xmin, ymax, xmax, labelName = tmpData

#                             if not labelName.__contains__("/"):
#                                 print('----------------')
#                                 continue
#                             print(doc.createElement("xmin"))
                            objectObj = doc.createElement("object")
                            rootNode.appendChild(objectObj)

                            objectName = doc.createElement("name")
                            objectNameText = doc.createTextNode(labelName)
                            objectName.appendChild(objectNameText)
                            objectObj.appendChild(objectName)

                            objectBndBox = doc.createElement("bndbox")
                            objectObj.appendChild(objectBndBox)

                            objectBndBoxXmin = doc.createElement("xmin")
                            objectBndBoxYmin = doc.createElement("ymin")
                            objectBndBoxXmax = doc.createElement("xmax")
                            objectBndBoxYmax = doc.createElement("ymax")

                            objectBndBoxXminText = doc.createTextNode(xmin)
                            objectBndBoxYminText = doc.createTextNode(ymin)
                            objectBndBoxXmaxText = doc.createTextNode(xmax)
                            objectBndBoxYmaxText = doc.createTextNode(ymax)

                            objectBndBox.appendChild(objectBndBoxXmin)
                            objectBndBox.appendChild(objectBndBoxYmin)
                            objectBndBox.appendChild(objectBndBoxXmax)
                            objectBndBox.appendChild(objectBndBoxYmax)

                            objectBndBoxXmin.appendChild(objectBndBoxXminText)
                            objectBndBoxYmin.appendChild(objectBndBoxYminText)
                            objectBndBoxXmax.appendChild(objectBndBoxXmaxText)
                            objectBndBoxYmax.appendChild(objectBndBoxYmaxText)

                            objectObj.appendChild(objectBndBox)
                        else:
                            continue
                    # save xml
                    xmlName=os.path.splitext(k)[0]+".xml"
                    targetPath = os.path.join(self._targetPath, xmlName)
                    with open(targetPath, mode="w", encoding=fileEncoding,errors="ignore") as fw:
                        doc.writexml(fw, indent="\t", newl="\n", addindent="\t", encoding=fileEncoding)
            except Exception as ex:
#                 MyLogger().error(f"OperateXML:Save xml error\n{ex}")
                print(f"OperateXML:Save xml error\n{ex}")
                return

In [3]:
#释放窗口
def releaseWin():
    cv.waitKey(0)
    cv.destroyAllWindows()
    key = cv.waitKey(0)
    if key==27:
        print(key)
        cv.destroyAllWindows()
        
def readPath(path):
    img_list, gray_list, name_list = [], [], []
    for name in os.listdir(path):
        img = cv.imread(path+r'\\'+name)
#         print(img.shape)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # 转为灰度图像
        img_list.append(img)
        gray_list.append(gray)
        name_list.append(name)
    return img_list, gray_list, name_list


# 二分迭代法-返回二值图像
def IterationMethod(img_gray, epsilon=0.0001, max_iter=1000):
    # 必须输入灰度图像
    assert img_gray.ndim == 2, "must input a gray image!"
    # 绘制灰度分布柱形图
#     pd.Series(img_gray.reshape(-1)).hist(bins=150)
#     plt.show()
    
    img_array = img_gray.reshape(-1)
    maxn, minn = img_array.max(), img_array.min()
    Tk, Tk1 = 0, (int(maxn)+int(minn)) / 2.0  # 初始化阈值
    it = 0
    begin = time.time()
    while abs(Tk1-Tk) > epsilon and it <= max_iter:
        Tk = Tk1
        Z_F = img_array[img_array > Tk]
        Z_B = img_array[img_array <= Tk]
        Tk1 = (sum(Z_F)/len(Z_F) + sum(Z_B)/len(Z_B)) / 2.0
        it += 1
    end = time.time()
    print(end-begin)
    # 得到最终的二值图像
    final_img = img_gray.copy()
    final_img[img_gray > Tk1] = 255
    final_img[img_gray <= Tk1] = 0
    return Tk1, final_img

def preprocessing(img, blur):
    """
        可调参数：高斯算子的大小blur
    """
    # 高斯模糊
    img = cv.GaussianBlur(img, (blur,blur), 0)
    img_copy = img.copy()
    print(img.shape)
    # 转为灰度图像
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # 二值化
    BestThresold, bin_img = IterationMethod(gray_img)
    return img_copy, gray_img, bin_img

# canny边缘检测，使用形态学的开运算和闭运算合并边缘围起来的区域
def edgeDetection2(bin_img, thres1, thres2, k):
    """
        开运算和闭运算 算子的大小
    """
    edge_img = cv.Canny(bin_img, thres1, thres2)
#     cv.imshow('edge_img', edge_img)
    x = cv.Sobel(edge_img, cv.CV_16S, 1, 0,ksize=5)
    y = cv.Sobel(edge_img, cv.CV_16S, 0, 1,ksize=3)
    Scale_absX = cv.convertScaleAbs(x)  # convert 转换  scale 缩放
    Scale_absY = cv.convertScaleAbs(y)
#     cv.imshow('Scale_absX', Scale_absX)
#     cv.imshow('Scale_absY', Scale_absY)
    
    kernel = np.ones((k ,k), np.uint8)
    edge_img1 = cv.morphologyEx(Scale_absX, cv.MORPH_CLOSE, kernel)
    edge_img2 = cv.morphologyEx(edge_img1, cv.MORPH_OPEN, kernel)
#     cv.imshow('edge_img1', edge_img1)
#     cv.imshow('edge_img2', edge_img2)
    return edge_img1, edge_img2

# 框出矩形区域
def Rectangle(img_copy, img_edge2):
    Min_Area = 1200
    Max_Area = 3000
    contours, hierarchy = cv.findContours(img_edge2, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = [cnt for cnt in contours if cv.contourArea(cnt) > Min_Area and cv.contourArea(cnt) < Max_Area]
    print('边框所围区域大于最小面积的边框个数', len(contours))
    
    car_contours = []
    for cnt in contours:
        rect = cv.minAreaRect(cnt) #求出边缘的最小外切矩形，rect包含：中心点，（长边，短边），旋转角度
        area_width, area_height = rect[1]
        if area_width < area_height:
            area_width, area_height = area_height, area_width
        wh_ratio = area_width / area_height 
        if wh_ratio > 3.0 and wh_ratio < 8.0:
            car_contours.append(rect)
            box = cv.boxPoints(rect)
            box = np.int0(box)
#             print("宽高比：",wh_ratio)
            cv.drawContours(img_copy, [box], 0, (0, 0, 255), 2)
    return car_contours

def adjust(box): # input: list(4*2)
    eps = 2
    xmin = sum(sorted(box[:,0])[:2])//2 - eps
    ymin = sum(sorted(box[:,1])[:2])//2 - eps
    xmax = sum(sorted(box[:,0])[2:])//2 + eps
    ymax = sum(sorted(box[:,1])[2:])//2 + eps
    return [xmin,ymin,xmax,ymax]

def generate_str(name, point):
    strr = name
    tmp = [point[1],point[0],point[3],point[2]]
    for p in tmp:
        strr += (','+str(p))
    strr += ',equation\n'
    return strr

In [4]:
def semi_auto_label(image_path, labeled_path, location_path):
    img_list, gray_list, name_list = readPath(image_path)
    for ind,img in enumerate(img_list):

        # 1、切割等式区域，转换为统一大小比例
        img = cv.resize(img,(600,800),interpolation=cv.INTER_CUBIC)
    #     [height,width,pixels] = img.shape
    #     cv.imshow('img',img)
        cv.imwrite(labeled_path+r'\\'+name_list[ind], img)
        img_cc = img.copy()
        # 2、矫正倾斜图片
        # 拍摄时注意一下，此操作可以省略

        # 3、去噪、二值化、形态学-边缘检测+闭运算+开运算，得到轮廓
        img_copy, gray_img, bin_img = preprocessing(img, 1)
    #     cv.imshow('gray_img',gray_img)
    #     cv.imshow('bin_img',bin_img)
        # 边缘检测
        edge_img1, edge_img2 = edgeDetection2(bin_img, 10, 100, 20)
        car_contours = Rectangle(img_copy, edge_img1)
        boxes = []
        for i in car_contours:
            box = cv.boxPoints(i)
            box = np.int0(box)
            tmp = box
            boxes.append(adjust(box))

        # 4、框定所有式子的位置，生成txt文件用于保存location信息
        txtContent = []
        for box in boxes:
            cv.rectangle(img_cc, (box[0],box[1]), (box[2],box[3]), (0,0,255), 2)
            txtContent.append(generate_str(name_list[ind],box))
        cv.imshow('img_cc',img_cc)        
        releaseWin()
        print(txtContent)
#         f = open(r'C:\Users\admin\Desktop\Data\DeepLearning_Data\data labeling\location\\'+name_list[ind].replace('.jpg','')+'.txt','a')
        f = open(location_path+r'\\'+name_list[ind].replace('.jpg','')+'.txt','a')
        f.seek(0)
        f.truncate()          # 清空内容再写入
        f.writelines(txtContent)
        f.close()

        # 5、生成xml文件
        srcName = name_list[ind].replace('.jpg','')+'.txt'
        operateXML = OpeateXML(location_path, labeled_path, srcName)
        a = operateXML.readSrcFileName()
        testData = name_list[ind]
        res=operateXML.getCreateXMLData(a,testData)
        operateXML.operateXML(res)

In [6]:
image_path = r"C:\Users\C1  H2  K3\Desktop\label\image"  # 存放原始图像的路径
labeled_path = r"C:\Users\C1  H2  K3\Desktop\label\labeled_data"  # 存放标记后图像的路径
location_path = r"C:\Users\C1  H2  K3\Desktop\label\location"  # 存放标记信息的路径

semi_auto_label(image_path, labeled_path, location_path)

(800, 600, 3)
0.37408018112182617
边框所围区域大于最小面积的边框个数 8
['1.jpg,691,192,718,306,equation\n', '1.jpg,641,353,669,463,equation\n', '1.jpg,599,350,623,464,equation\n', '1.jpg,477,348,499,461,equation\n']
(800, 600, 3)
0.2143557071685791
边框所围区域大于最小面积的边框个数 42
['10.jpg,760,243,783,353,equation\n', '10.jpg,714,224,747,377,equation\n', '10.jpg,702,45,737,186,equation\n', '10.jpg,661,226,690,366,equation\n', '10.jpg,655,47,680,176,equation\n', '10.jpg,615,229,640,355,equation\n', '10.jpg,606,47,633,178,equation\n', '10.jpg,576,408,605,563,equation\n', '10.jpg,567,229,594,368,equation\n', '10.jpg,559,51,586,200,equation\n', '10.jpg,518,230,545,377,equation\n', '10.jpg,513,52,538,201,equation\n', '10.jpg,471,233,498,374,equation\n', '10.jpg,467,54,495,195,equation\n', '10.jpg,428,409,457,559,equation\n', '10.jpg,424,235,453,380,equation\n', '10.jpg,421,56,449,203,equation\n', '10.jpg,379,410,408,553,equation\n', '10.jpg,379,235,404,378,equation\n', '10.jpg,377,57,402,209,equation\n', '10.jpg,335,23

0.26632189750671387
边框所围区域大于最小面积的边框个数 24
['17.jpg,715,183,739,311,equation\n', '17.jpg,674,352,697,464,equation\n', '17.jpg,673,185,699,300,equation\n', '17.jpg,630,354,655,461,equation\n', '17.jpg,628,184,654,296,equation\n', '17.jpg,588,188,613,312,equation\n', '17.jpg,586,354,612,463,equation\n', '17.jpg,546,353,569,456,equation\n', '17.jpg,544,186,569,306,equation\n', '17.jpg,505,353,529,470,equation\n', '17.jpg,503,187,527,306,equation\n', '17.jpg,463,353,486,469,equation\n', '17.jpg,461,188,485,304,equation\n', '17.jpg,423,353,445,450,equation\n', '17.jpg,381,354,403,477,equation\n', '17.jpg,339,353,365,496,equation\n', '17.jpg,298,355,322,455,equation\n', '17.jpg,258,355,281,502,equation\n', '17.jpg,257,190,280,305,equation\n', '17.jpg,215,355,241,465,equation\n', '17.jpg,217,194,239,306,equation\n', '17.jpg,177,356,200,483,equation\n', '17.jpg,135,355,157,475,equation\n']
(800, 600, 3)
0.21838140487670898
边框所围区域大于最小面积的边框个数 32
['18.jpg,744,233,767,337,equation\n', '18.jpg,697,21

0.22243738174438477
边框所围区域大于最小面积的边框个数 41
['24.jpg,771,263,801,390,equation\n', '24.jpg,721,239,751,394,equation\n', '24.jpg,713,33,740,156,equation\n', '24.jpg,676,461,705,601,equation\n', '24.jpg,668,245,696,383,equation\n', '24.jpg,666,33,695,174,equation\n', '24.jpg,624,458,650,588,equation\n', '24.jpg,619,244,645,367,equation\n', '24.jpg,616,37,642,165,equation\n', '24.jpg,576,457,600,575,equation\n', '24.jpg,570,246,596,389,equation\n', '24.jpg,564,38,592,191,equation\n', '24.jpg,527,457,554,579,equation\n', '24.jpg,523,248,548,384,equation\n', '24.jpg,519,39,543,180,equation\n', '24.jpg,481,459,507,580,equation\n', '24.jpg,475,251,500,367,equation\n', '24.jpg,472,44,497,198,equation\n', '24.jpg,435,458,459,582,equation\n', '24.jpg,429,254,460,373,equation\n', '24.jpg,427,50,452,199,equation\n', '24.jpg,390,456,415,590,equation\n', '24.jpg,386,258,412,395,equation\n', '24.jpg,380,54,405,196,equation\n', '24.jpg,347,458,373,585,equation\n', '24.jpg,343,259,368,378,equation\n', '24.

0.435833215713501
边框所围区域大于最小面积的边框个数 14
['3.jpg,723,214,745,313,equation\n', '3.jpg,677,196,704,303,equation\n', '3.jpg,632,197,658,313,equation\n', '3.jpg,586,365,610,470,equation\n', '3.jpg,584,199,612,336,equation\n', '3.jpg,548,38,573,152,equation\n', '3.jpg,544,202,569,321,equation\n', '3.jpg,503,39,530,153,equation\n', '3.jpg,501,202,529,313,equation\n', '3.jpg,461,358,483,471,equation\n']
(800, 600, 3)
0.26833105087280273
边框所围区域大于最小面积的边框个数 36
['30.jpg,744,435,767,528,equation\n', '30.jpg,742,254,766,371,equation\n', '30.jpg,703,428,729,590,equation\n', '30.jpg,695,236,723,391,equation\n', '30.jpg,684,50,711,207,equation\n', '30.jpg,650,430,679,583,equation\n', '30.jpg,646,236,674,381,equation\n', '30.jpg,636,54,663,196,equation\n', '30.jpg,602,428,633,564,equation\n', '30.jpg,596,239,624,403,equation\n', '30.jpg,590,57,615,179,equation\n', '30.jpg,555,426,582,565,equation\n', '30.jpg,549,241,579,393,equation\n', '30.jpg,546,58,571,195,equation\n', '30.jpg,504,425,532,564,equation

边框所围区域大于最小面积的边框个数 31
['37.jpg,762,259,785,371,equation\n', '37.jpg,711,229,737,313,equation\n', '37.jpg,706,45,736,155,equation\n', '37.jpg,663,228,692,339,equation\n', '37.jpg,659,48,686,172,equation\n', '37.jpg,620,410,647,566,equation\n', '37.jpg,616,231,643,358,equation\n', '37.jpg,612,51,639,190,equation\n', '37.jpg,570,231,596,344,equation\n', '37.jpg,567,53,591,198,equation\n', '37.jpg,524,231,550,364,equation\n', '37.jpg,522,55,548,179,equation\n', '37.jpg,432,233,458,376,equation\n', '37.jpg,433,59,459,200,equation\n', '37.jpg,389,234,415,336,equation\n', '37.jpg,390,62,414,205,equation\n', '37.jpg,343,235,369,355,equation\n', '37.jpg,301,236,324,360,equation\n', '37.jpg,261,68,284,173,equation\n', '37.jpg,258,236,283,362,equation\n', '37.jpg,253,409,280,566,equation\n', '37.jpg,220,69,242,194,equation\n', '37.jpg,213,238,240,356,equation\n', '37.jpg,209,408,235,549,equation\n', '37.jpg,177,70,202,203,equation\n', '37.jpg,173,239,197,387,equation\n', '37.jpg,135,71,159,199,equ

0.22742128372192383
边框所围区域大于最小面积的边框个数 30
['44.jpg,744,245,767,360,equation\n', '44.jpg,695,223,724,384,equation\n', '44.jpg,649,223,677,362,equation\n', '44.jpg,644,413,668,549,equation\n', '44.jpg,607,30,633,142,equation\n', '44.jpg,603,223,626,358,equation\n', '44.jpg,597,414,622,546,equation\n', '44.jpg,560,32,587,187,equation\n', '44.jpg,555,225,580,333,equation\n', '44.jpg,554,412,578,546,equation\n', '44.jpg,513,34,539,164,equation\n', '44.jpg,513,226,537,336,equation\n', '44.jpg,511,413,534,543,equation\n', '44.jpg,466,226,491,374,equation\n', '44.jpg,467,36,497,174,equation\n', '44.jpg,464,411,491,527,equation\n', '44.jpg,425,411,447,536,equation\n', '44.jpg,423,39,448,175,equation\n', '44.jpg,422,229,446,358,equation\n', '44.jpg,383,412,406,523,equation\n', '44.jpg,381,228,404,350,equation\n', '44.jpg,341,409,365,531,equation\n', '44.jpg,338,228,362,350,equation\n', '44.jpg,299,408,322,518,equation\n', '44.jpg,296,229,320,342,equation\n', '44.jpg,293,45,318,170,equation\n', '4

0.21527457237243652
边框所围区域大于最小面积的边框个数 33
['50.jpg,756,236,778,352,equation\n', '50.jpg,714,14,741,130,equation\n', '50.jpg,710,201,737,341,equation\n', '50.jpg,666,15,695,145,equation\n', '50.jpg,622,-3,652,153,equation\n', '50.jpg,581,16,604,151,equation\n', '50.jpg,534,16,559,147,equation\n', '50.jpg,531,199,556,315,equation\n', '50.jpg,491,17,515,129,equation\n', '50.jpg,486,382,509,529,equation\n', '50.jpg,487,199,513,355,equation\n', '50.jpg,447,17,471,153,equation\n', '50.jpg,444,199,469,356,equation\n', '50.jpg,399,201,424,348,equation\n', '50.jpg,398,18,425,145,equation\n', '50.jpg,358,201,382,339,equation\n', '50.jpg,358,21,382,139,equation\n', '50.jpg,356,380,380,541,equation\n', '50.jpg,315,199,339,345,equation\n', '50.jpg,314,380,338,518,equation\n', '50.jpg,314,18,339,144,equation\n', '50.jpg,273,199,295,324,equation\n', '50.jpg,272,21,296,138,equation\n', '50.jpg,231,199,253,305,equation\n', '50.jpg,229,19,252,128,equation\n', '50.jpg,188,198,213,324,equation\n', '50.jpg,

0.15957307815551758
边框所围区域大于最小面积的边框个数 23
['57.jpg,754,260,776,369,equation\n', '57.jpg,634,421,658,535,equation\n', '57.jpg,625,241,648,345,equation\n', '57.jpg,620,57,645,204,equation\n', '57.jpg,590,423,615,557,equation\n', '57.jpg,582,241,609,392,equation\n', '57.jpg,545,421,570,573,equation\n', '57.jpg,501,420,524,568,equation\n', '57.jpg,490,62,516,187,equation\n', '57.jpg,457,423,482,570,equation\n', '57.jpg,451,243,478,379,equation\n', '57.jpg,447,65,473,190,equation\n', '57.jpg,412,245,434,358,equation\n', '57.jpg,325,424,353,565,equation\n', '57.jpg,202,73,227,178,equation\n', '57.jpg,198,248,225,382,equation\n', '57.jpg,195,426,219,574,equation\n', '57.jpg,163,74,185,174,equation\n', '57.jpg,158,247,181,367,equation\n', '57.jpg,150,426,176,563,equation\n', '57.jpg,111,250,142,381,equation\n', '57.jpg,108,428,134,569,equation\n']
(800, 600, 3)
0.214461088180542
边框所围区域大于最小面积的边框个数 36
['58.jpg,735,37,765,145,equation\n', '58.jpg,732,217,761,338,equation\n', '58.jpg,688,35,714,147

['9.jpg,761,213,783,320,equation\n', '9.jpg,717,192,742,338,equation\n', '9.jpg,669,18,695,150,equation\n', '9.jpg,622,195,646,308,equation\n', '9.jpg,623,18,647,124,equation\n', '9.jpg,619,369,646,518,equation\n', '9.jpg,574,196,600,328,equation\n', '9.jpg,573,21,600,141,equation\n', '9.jpg,572,369,599,511,equation\n', '9.jpg,529,198,554,312,equation\n', '9.jpg,528,23,554,154,equation\n', '9.jpg,527,369,553,448,equation\n', '9.jpg,484,369,510,520,equation\n', '9.jpg,484,201,509,331,equation\n', '9.jpg,482,25,507,163,equation\n', '9.jpg,438,369,463,509,equation\n', '9.jpg,437,201,462,323,equation\n', '9.jpg,436,27,462,156,equation\n', '9.jpg,395,203,420,339,equation\n', '9.jpg,394,368,419,505,equation\n', '9.jpg,392,31,417,177,equation\n', '9.jpg,352,368,376,501,equation\n', '9.jpg,351,203,376,329,equation\n', '9.jpg,306,206,332,346,equation\n', '9.jpg,265,203,288,346,equation\n', '9.jpg,98,209,124,345,equation\n']
